In [ ]:
# PEG backtesting

In [35]:
# os.chdir(r"C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\Muut\DataAnalysis\Investing\models")

In [ ]:
print(os.getcwd())

In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [ ]:
# Call the main function and store the returned DataFrame
df = main()

In [ ]:
# Show dataframe
print(df.head())

## Backtesting model 1
Condition 1: The portfolio cash must be at least 1,000 for a buy to occur. 
Condition 2: You can only hold shares of one ticker at a time (current_symbol keeps track of the current stock).
Condition 3: The backtest enforces a rule where a new buy cannot occur until the previous position is sold. The next trade can only happen on or after the sell date.
Condition 4: The backtesting runs until the data is exhausted (the last date in the dataset).


In [12]:
# Backtesting 1

# Initial setup
initial_cash = 10000
portfolio_cash = initial_cash
portfolio_stock = 0  # initially no stock
current_symbol = None  # track current symbol in the portfolio
buy_price = 0
trade_fee_percent = 0.0  # 0.1%
profit_target = 1.10  # 10% profit target
stop_loss = 0.95  # 5% stop loss
alert_column = 'pegAlert'  # column in your DataFrame with 1s and 0s for potential trading opportunity

# Metrics initialization
total_trades = 0
total_profit = 0
max_profit = 0
max_drawdown = 0
peak_value = initial_cash

# Condition constants
min_cash_to_trade = 1000
last_trade_date = None  # track the date of the last trade (sell)

# Assuming your DataFrame has columns: 'date', 'ticker', 'close', 'alert' and it's sorted by date for each ticker
def backtest(df):
    global portfolio_cash, portfolio_stock, buy_price, total_trades, total_profit, max_profit, max_drawdown, peak_value, current_symbol, last_trade_date
    
    trades = []  # To store trade data (buy/sell)
    
    for index, row in df.iterrows():
        close_price = row['close']
        alert = row[alert_column]
        ticker = row['symbol']
        current_date = row['timestamp']
        
        # Sell condition (sell only if there's stock)
        if portfolio_stock > 0:
            sell_condition_profit = close_price >= buy_price * profit_target
            sell_condition_loss = close_price <= buy_price * stop_loss
            
            if sell_condition_profit or sell_condition_loss:
                total_value = portfolio_stock * close_price
                fee = total_value * trade_fee_percent
                total_value -= fee  # after fee
                
                # Sell the stock
                profit = total_value - (portfolio_stock * buy_price) - (portfolio_stock * buy_price * trade_fee_percent)
                portfolio_cash += total_value
                total_profit += profit
                portfolio_stock = 0
                last_trade_date = current_date  # update last trade date
                current_symbol = None  # clear the current symbol after selling
                
                trades.append({'date': current_date, 'symbol': ticker, 'type': 'sell', 'price': close_price, 'shares': portfolio_stock, 'fee': fee})
                
                # Calculate max profit and drawdown
                current_value = portfolio_cash
                if current_value > peak_value:
                    peak_value = current_value
                drawdown = (peak_value - current_value) / peak_value
                max_drawdown = max(max_drawdown, drawdown)
                max_profit = max(max_profit, profit)
        
        # Buy condition: Alert is 1, cash >= 1000, no current symbol, and chronological (next buy after last sell)
        if alert == 1 and portfolio_cash >= min_cash_to_trade and portfolio_stock == 0 and (last_trade_date is None or current_date >= last_trade_date) and current_symbol is None:
            # Calculate the number of shares to buy
            shares_to_buy = portfolio_cash / close_price
            total_cost = shares_to_buy * close_price
            fee = total_cost * trade_fee_percent
            total_cost += fee  # including fee
            
            # Buy the stock
            portfolio_stock += shares_to_buy
            portfolio_cash -= total_cost
            buy_price = close_price
            current_symbol = ticker  # set the current symbol
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'buy', 'price': close_price, 'shares': shares_to_buy, 'fee': fee})
            total_trades += 1
    
    # Calculate final portfolio value and returns
    final_portfolio_value = portfolio_cash
    total_return = (final_portfolio_value - initial_cash) / initial_cash * 100  # in %
    
    return {
        'trades': trades,
        'final_portfolio_value': final_portfolio_value,
        'total_return': total_return,
        'total_profit': total_profit,
        'total_trades': total_trades,
        'max_profit': max_profit,
        'max_drawdown': max_drawdown
    }

# Example DataFrame setup
# df = pd.DataFrame({
#     'timestamp': [1,2,3,4,5,6,7,8,9,10],  # replace with your data
#     'symbol': ['AA','AB','AA','AB','AA','AA','AB','AA','AB','AA'],  # replace with your data
#     'close': [10,100,11,110,12,13,120,14,130,15],  # replace with your data
#     'pegAlert': [0,1,0,0,0,0,0,0,0,0],  # replace with your calculated alerts
# })

df = pd.DataFrame({
    'timestamp': [1,2,3,4,5,6,7,8,9,10],  # replace with your data
    'symbol': ['AA','AA','AA','AA','AA','AA','AA','AA','AA','AA'],  # replace with your data
    'close': [10,11,11,12,13,13,12,11,13,15],  # replace with your data
    'pegAlert': [0,1,0,0,0,0,1,0,0,0],  # replace with your calculated alerts
})

# Run backtest
results = backtest(df)

# Convert trades to DataFrame for analysis
trades_df = pd.DataFrame(results['trades'])

# Output results
print(f"Final portfolio cash: {results['final_portfolio_value']}")
print(f"Total return: {results['total_return']}%")
print(f"Total profit: {results['total_profit']}")
print(f"Total trades: {results['total_trades']}")
print(f"Max profit from a single trade: {results['max_profit']}")
print(f"Max drawdown: {results['max_drawdown'] * 100}%")


Final portfolio cash: 10833.333333333334
Total return: 8.33333333333334%
Total profit: 833.3333333333339
Total trades: 2
Max profit from a single trade: 1818.181818181818
Max drawdown: 8.333333333333327%


In [15]:
# Model 1 edited version

# Initial setup
initial_cash = 10000
portfolio_cash = initial_cash
portfolio_stock = 0  # initially no stock
current_symbol = None  # track current symbol in the portfolio
buy_price = 0
trade_fee_percent = 0.00  # 0.1%
profit_target = 1.10  # 10% profit target
stop_loss = 0.95  # 5% stop loss
alert_column = 'pegAlert'  # column in your DataFrame with 1s and 0s for potential trading opportunity

# Metrics initialization
total_trades = 0
total_profit = 0
max_profit = 0
max_drawdown = 0
peak_value = initial_cash
total_fees = 0

# Trade records
trades = []
win_trades = 0
trade_durations = []

# Condition constants
min_cash_to_trade = 1000
last_trade_date = None  # track the date of the last trade (sell)

# Buy condition function
def buy_condition(alert, close_price, portfolio_cash, current_symbol, last_trade_date, current_date):
    return alert == 1 and portfolio_cash >= min_cash_to_trade and portfolio_stock == 0 and \
           (last_trade_date is None or current_date >= last_trade_date) and current_symbol is None

# Sell condition function
def sell_condition(close_price, buy_price):
    sell_condition_profit = close_price >= buy_price * profit_target
    sell_condition_loss = close_price <= buy_price * stop_loss
    return sell_condition_profit or sell_condition_loss

# Backtesting function
def backtest(df):
    global portfolio_cash, portfolio_stock, buy_price, total_trades, total_profit, max_profit, max_drawdown, peak_value, current_symbol, last_trade_date, total_fees, win_trades, trade_durations
    
    for index, row in df.iterrows():
        close_price = row['close']
        alert = row[alert_column]
        ticker = row['symbol']
        current_date = row['timestamp']
        
        # Sell condition (sell only if there's stock)
        if portfolio_stock > 0 and sell_condition(close_price, buy_price):
            total_value = portfolio_stock * close_price
            fee = total_value * trade_fee_percent
            total_value -= fee  # after fee
            total_fees += fee  # accumulate total fees
            
            # Sell the stock
            profit = total_value - (portfolio_stock * buy_price) - (portfolio_stock * buy_price * trade_fee_percent)
            portfolio_cash += total_value
            total_profit += profit
            trade_duration = (current_date - last_trade_date).days  # Track trade duration
            trade_durations.append(trade_duration)
            
            if profit > 0:
                win_trades += 1
            
            portfolio_stock = 0
            last_trade_date = current_date
            current_symbol = None
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'sell', 'price': close_price, 'shares': portfolio_stock, 'fee': fee, 'profit': profit, 'duration': trade_duration})
            
            # Calculate max profit and drawdown
            current_value = portfolio_cash
            if current_value > peak_value:
                peak_value = current_value
            drawdown = (peak_value - current_value) / peak_value
            max_drawdown = max(max_drawdown, drawdown)
            max_profit = max(max_profit, profit)
        
        # Buy condition
        if buy_condition(alert, close_price, portfolio_cash, current_symbol, last_trade_date, current_date):
            shares_to_buy = portfolio_cash / close_price
            total_cost = shares_to_buy * close_price
            fee = total_cost * trade_fee_percent
            total_cost += fee  # including fee
            total_fees += fee  # accumulate total fees
            
            # Buy the stock
            portfolio_stock += shares_to_buy
            portfolio_cash -= total_cost
            buy_price = close_price
            current_symbol = ticker
            last_trade_date = current_date
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'buy', 'price': close_price, 'shares': shares_to_buy, 'fee': fee})
            total_trades += 1
    
    # Calculate final portfolio value and returns
    final_portfolio_value = portfolio_cash
    total_return = round((final_portfolio_value - initial_cash) / initial_cash * 100, 2)  # in %
    
    # Calculate win rate
    win_rate = round(win_trades / total_trades * 100, 2) if total_trades > 0 else 0
    
    # Calculate best and worst trades (returns in %)
    trade_returns = [(t['profit'] / (t['shares'] * buy_price)) * 100 
                 for t in trades if t['type'] == 'sell' and t['shares'] > 0 and buy_price > 0]
    best_trade = round(max(trade_returns, default=0), 2)
    worst_trade = round(min(trade_returns, default=0), 2)
    
    # Average trade duration
    avg_duration = round(sum(trade_durations) / len(trade_durations), 2) if trade_durations else 0
    
    return {
        'trades': trades,
        'final_portfolio_value': round(final_portfolio_value, 2),
        'total_return': total_return,
        'total_profit': round(total_profit, 2),
        'total_trades': total_trades,
        'total_fees': round(total_fees, 2),
        'win_rate': win_rate,
        'best_trade': best_trade,
        'worst_trade': worst_trade,
        'avg_trade_duration': avg_duration,
        'max_profit': round(max_profit, 2),
        'max_drawdown': round(max_drawdown * 100, 2)  # in %
    }

# Example DataFrame setup
df = pd.DataFrame({
    'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
    'symbol': ['AA','AA','AA','AA','AA','AA','AA','AA','AA','AA'],  
    'close': [10,11,11,12,13,13,12,11,13,15],  
    'pegAlert': [0,1,0,0,0,0,1,0,0,0],  
})

# Run backtest
results = backtest(df)

# Convert trades to DataFrame for analysis
trades_df = pd.DataFrame(results['trades'])

# Output results
print(f"Final portfolio cash: {results['final_portfolio_value']}")
print(f"Total return: {results['total_return']}%")
print(f"Total profit: {results['total_profit']}")
print(f"Total trades: {results['total_trades']}")
print(f"Total fees: {results['total_fees']}")
print(f"Win rate: {results['win_rate']}%")
print(f"Best trade: {results['best_trade']}%")
print(f"Worst trade: {results['worst_trade']}%")
print(f"Average trade duration: {results['avg_trade_duration']} days")
print(f"Max profit from a single trade: {results['max_profit']}")
print(f"Max drawdown: {results['max_drawdown']}%")


Final portfolio cash: 10833.33
Total return: 8.33%
Total profit: 833.33
Total trades: 2
Total fees: 0.0
Win rate: 50.0%
Best trade: 0%
Worst trade: 0%
Average trade duration: 2.0 days
Max profit from a single trade: 1818.18
Max drawdown: 8.33%


In [16]:
print(trades_df.head())

        date symbol  type  price      shares  fee       profit  duration
0 2022-01-03     AA   buy     11  909.090909  0.0          NaN       NaN
1 2022-01-06     AA  sell     13    0.000000  0.0  1818.181818       3.0
2 2022-01-08     AA   buy     12  984.848485  0.0          NaN       NaN
3 2022-01-09     AA  sell     11    0.000000  0.0  -984.848485       1.0
